In [1]:
#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/HYBRID-Imbalanced']
projects=project.get_experiments()



def get_results(model, PARAMS):
    X_test,Y_test = instead_data_loaders(PARAMS,'test')

    X_test_pcg = X_test[:,:,:,0:3]/255.0
    X_test_ecg = X_test[:,:,:,3:6]/255.0
    
    y_pred = model.predict([X_test_pcg,X_test_ecg], batch_size=X_test.shape[0], verbose=1)
    y_test = np.argmax(Y_test,axis=1)
    
    print(custom_metrics._class_metrics(y_test, y_pred[:, 1] > threshold))
    return y_test, y_pred
    


def neptune_log_results(y_test=None, y_pred=None, threshold = 0.5):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)
    

    

In [ ]:
for i in range(len(projects)):
    if i==5:break
        
    project    = projects[i]
    model_name = str(project)+'.h5'
    PARAMS     = project.get_parameters()
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    
    y_test, y_pred=get_results(model, PARAMS)
    
    #neptune.init('intelliscope/Test-HYBRID-Imbalanced',NEPTUNE_API_TOKEN)
    #exp = neptune.create_experiment(name=str(project),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    #neptune_log_metrics(y_test, y_pred)
    #neptune.stop()

Total  707
199 Images loaded across 2 Categories. Ignored blanks  60
508 Images loaded across 2 Categories. Ignored blanks  9
638 Total After removing blanks
